In [ ]:
-- LAB | SQL Subqueries - Sakila Database
-- Solutions to all challenges

-- =====================================================
-- MAIN CHALLENGES
-- =====================================================

-- 1. Determine the number of copies of the film "Hunchback Impossible" that exist in the inventory system.

SELECT COUNT(*) as number_of_copies
FROM inventory i
JOIN film f ON i.film_id = f.film_id
WHERE f.title = 'Hunchback Impossible';

-- Alternative using subquery:
SELECT COUNT(*) as number_of_copies
FROM inventory
WHERE film_id = (
    SELECT film_id 
    FROM film 
    WHERE title = 'Hunchback Impossible'
);


-- 2. List all films whose length is longer than the average length of all the films in the Sakila database.

SELECT title, length
FROM film
WHERE length > (
    SELECT AVG(length) 
    FROM film
)
ORDER BY length DESC;


-- 3. Use a subquery to display all actors who appear in the film "Alone Trip".

SELECT a.first_name, a.last_name
FROM actor a
WHERE a.actor_id IN (
    SELECT fa.actor_id
    FROM film_actor fa
    JOIN film f ON fa.film_id = f.film_id
    WHERE f.title = 'Alone Trip'
);


-- =====================================================
-- BONUS CHALLENGES
-- =====================================================

-- 1. Sales have been lagging among young families, and you want to target family movies for a promotion. 
-- Identify all movies categorized as family films.

SELECT f.title
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
WHERE c.name = 'Family'
ORDER BY f.title;

-- Alternative using subquery:
SELECT title
FROM film
WHERE film_id IN (
    SELECT fc.film_id
    FROM film_category fc
    JOIN category c ON fc.category_id = c.category_id
    WHERE c.name = 'Family'
)
ORDER BY title;


-- 2. Retrieve the name and email of customers from Canada using both subqueries and joins.

-- Using JOINS:
SELECT c.first_name, c.last_name, c.email
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN country co ON ci.country_id = co.country_id
WHERE co.country = 'Canada';

-- Using SUBQUERIES:
SELECT first_name, last_name, email
FROM customer
WHERE address_id IN (
    SELECT address_id
    FROM address
    WHERE city_id IN (
        SELECT city_id
        FROM city
        WHERE country_id = (
            SELECT country_id
            FROM country
            WHERE country = 'Canada'
        )
    )
);


-- 3. Determine which films were starred by the most prolific actor in the Sakila database.
-- First find the most prolific actor, then find their films.

-- Step 1: Find the most prolific actor
SELECT actor_id, COUNT(*) as film_count
FROM film_actor
GROUP BY actor_id
ORDER BY film_count DESC
LIMIT 1;

-- Step 2: Complete solution - Films starred by the most prolific actor
SELECT f.title
FROM film f
JOIN film_actor fa ON f.film_id = fa.film_id
WHERE fa.actor_id = (
    SELECT actor_id
    FROM film_actor
    GROUP BY actor_id
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
ORDER BY f.title;

-- With actor name included:
SELECT a.first_name, a.last_name, f.title
FROM film f
JOIN film_actor fa ON f.film_id = fa.film_id
JOIN actor a ON fa.actor_id = a.actor_id
WHERE fa.actor_id = (
    SELECT actor_id
    FROM film_actor
    GROUP BY actor_id
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
ORDER BY f.title;


-- 4. Find the films rented by the most profitable customer in the Sakila database.

-- First, find the most profitable customer:
SELECT customer_id, SUM(amount) as total_spent
FROM payment
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 1;

-- Complete solution - Films rented by most profitable customer:
SELECT DISTINCT f.title
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
WHERE r.customer_id = (
    SELECT customer_id
    FROM payment
    GROUP BY customer_id
    ORDER BY SUM(amount) DESC
    LIMIT 1
)
ORDER BY f.title;

-- With customer details:
SELECT c.first_name, c.last_name, f.title, 
       (SELECT SUM(amount) FROM payment WHERE customer_id = c.customer_id) as total_spent
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN customer c ON r.customer_id = c.customer_id
WHERE r.customer_id = (
    SELECT customer_id
    FROM payment
    GROUP BY customer_id
    ORDER BY SUM(amount) DESC
    LIMIT 1
)
ORDER BY f.title;


-- 5. Retrieve the client_id and the total_amount_spent of those clients who spent more than the average 
-- of the total_amount spent by each client.

-- First, let's see the average amount spent per customer:
SELECT AVG(customer_total) as avg_spent_per_customer
FROM (
    SELECT customer_id, SUM(amount) as customer_total
    FROM payment
    GROUP BY customer_id
) as customer_totals;

-- Complete solution:
SELECT customer_id, total_amount_spent
FROM (
    SELECT customer_id, SUM(amount) as total_amount_spent
    FROM payment
    GROUP BY customer_id
) as customer_spending
WHERE total_amount_spent > (
    SELECT AVG(customer_total)
    FROM (
        SELECT customer_id, SUM(amount) as customer_total
        FROM payment
        GROUP BY customer_id
    ) as avg_calculation
)
ORDER BY total_amount_spent DESC;

-- Alternative more readable version:
WITH customer_totals AS (
    SELECT customer_id, SUM(amount) as total_amount_spent
    FROM payment
    GROUP BY customer_id
),
average_spending AS (
    SELECT AVG(total_amount_spent) as avg_total
    FROM customer_totals
)
SELECT ct.customer_id, ct.total_amount_spent
FROM customer_totals ct
CROSS JOIN average_spending av
WHERE ct.total_amount_spent > av.avg_total
ORDER BY ct.total_amount_spent DESC;


-- =====================================================
-- ADDITIONAL VERIFICATION QUERIES
-- =====================================================

-- Verify film count and average length
SELECT 
    COUNT(*) as total_films,
    AVG(length) as average_length,
    MIN(length) as min_length,
    MAX(length) as max_length
FROM film;

-- Count of films longer than average
SELECT COUNT(*) as films_longer_than_average
FROM film
WHERE length > (SELECT AVG(length) FROM film);

-- Verify most prolific actor details
SELECT a.first_name, a.last_name, COUNT(*) as film_count
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY film_count DESC
LIMIT 5;